In [31]:
from utils import *
from pprint import pprint

In [69]:
house = importfile('hw3_house_votes_84.csv', ',')
housecategory = {}
for i in house[0]:
    housecategory[i] = 'categorical'
housecategory["class"] = 'class'
housedata = np.array(house[1:]).astype(float)

# print(len(housedata))
# print(housecategory)

#class in last column

435
{'#handicapped-infants': 'categorical', 'water-project-cost-sharing': 'categorical', 'adoption-of-the-budget-resolution': 'categorical', 'physician-fee-freeze': 'categorical', 'el-salvador-adi': 'categorical', 'religious-groups-in-schools': 'categorical', 'anti-satellite-test-ban': 'categorical', 'aid-to-nicaraguan-contras': 'categorical', 'mx-missile': 'categorical', 'immigration': 'categorical', 'synfuels-corporation-cutback': 'categorical', 'education-spending': 'categorical', 'superfund-right-to-sue': 'categorical', 'crime': 'categorical', 'duty-free-exports': 'categorical', 'export-administration-act-south-africa': 'categorical', 'class': 'class'}


In [71]:
wine = importfile('hw3_wine.csv', '\t')
winecategory = {}
for i in wine[0]:
    winecategory[i] = 'numerical'
winecategory["# class"] = 'class'
winedata = np.array(wine[1:]).astype(float)
# winedata = np.array([[float(item) for item in onelist] for onelist in wine[1:]])

# print(len(winedata))
# print(winecategory)

#class in first column

178
{'# class': 'class', '1': 'numerical', '2': 'numerical', '3': 'numerical', '4': 'numerical', '5': 'numerical', '6': 'numerical', '7': 'numerical', '8': 'numerical', '9': 'numerical', '10': 'numerical', '11': 'numerical', '12': 'numerical', '13': 'numerical'}


In [73]:
cancer = importfile('hw3_cancer.csv', '\t')
cancercategory = {}
for i in cancer[0]:
    cancercategory[i] = 'numerical'
cancercategory["Class"] = 'class'
cancerdata = np.array(cancer[1:]).astype(float)

#print(len(cancerdata))
#print(cancercategory)

#class in last column

{'Clump_Thickness': 'numerical', 'Cell_Size_Uniformity': 'numerical', 'Cell_Shape_Uniformity': 'numerical', 'Marginal_Adhesion': 'numerical', 'Single_Epi_Cell_Size': 'numerical', 'Bare_Nuclei': 'numerical', 'Bland_Chromatin': 'numerical', 'Normal_Nucleoli': 'numerical', 'Mitoses': 'numerical', 'Class': 'class'}


In [74]:
cmc = importfile('cmc.data', ',')
cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
"Husband's education":"categorical","Number of children ever born":"numerical",
"Wife's religion":"binary","Wife's now working?":"binary",
"Husband's occupation":"categorical","Standard-of-living index":"categorical",
"Media exposure":"binary","Contraceptive method used":"class"}
cmcdata = np.array(cmc).astype(int)

# print(len(cmcdata))
# print(cmccategory)

#class in last column

{"Wife's age": 'numerical', "Wife's education": 'categorical', "Husband's education": 'categorical', 'Number of children ever born': 'numerical', "Wife's religion": 'binary', "Wife's now working?": 'binary', "Husband's occupation": 'categorical', 'Standard-of-living index': 'categorical', 'Media exposure': 'binary', 'Contraceptive method used': 'class'}
